In [1]:
# Importing Potentially Used Basic Dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import sqlalchemy as db
from sqlalchemy import create_engine
import io
import psycopg2

In [2]:
from config import db_password, username

# Connect To PostgreSQL Database With SQLAlchemy

In [3]:
host = 'movies-fp.cpige012zhtw.us-east-1.rds.amazonaws.com'
port = 5432
passw = db_password
database = "postgres"
port=5432

   
db_string = "postgresql://" + username + ":" + passw + "@" + host + ":" + "5432/" + database
engine = db.create_engine(db_string)
connection = engine.connect()
metadata = db.MetaData()

# Query Database and Select Tables To Import and Create Dataframes

In [4]:
movie_genres = db.Table('movie_genre', metadata, autoload=True, autoload_with=engine)
query = db.select([movie_genres]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
mov_gen_df = pd.DataFrame(ResultSet)
mov_gen_df.columns = ResultSet[0].keys()
mov_gen_df = mov_gen_df.loc[:, ['movie_id', 'mg_id', 'genre_id']]

mov_gen_df.head()

,movie_id,mg_id,genre_id
0,1,0,1
1,1,1,3
2,1,2,4
3,1,3,5
4,1,4,9


In [5]:
movies = db.Table('movies', metadata, autoload=True, autoload_with=engine)
query = db.select([movies]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
movies_df = pd.DataFrame(ResultSet)
movies_df.columns = ResultSet[0].keys()
movies_df = movies_df.loc[:, ["movie_id", "name", "year", "runtime"]]

movies_df.head()

,movie_id,name,year,runtime
0,1,Toy Story,1995,81
1,2,Jumanji,1995,104
2,3,Grumpier Old Men,1995,101
3,4,Waiting to Exhale,1995,127
4,5,Father of the Bride Part II,1995,106


In [6]:
ratings = db.Table('ratings', metadata, autoload=True, autoload_with=engine)
query = db.select([ratings]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ratings_df = pd.DataFrame(ResultSet)
ratings_df.columns = ResultSet[0].keys()
ratings_df = ratings_df.loc[:, ['movie_id', 'user_id', 'rating']]

ratings_df.head()

,movie_id,user_id,rating
0,307,1,3.5
1,481,1,3.5
2,1091,1,1.5
3,1257,1,4.5
4,1449,1,4.5


In [7]:
# meta = db.Table('movies_metadata', metadata, autoload=True, autoload_with=engine)
# query = db.select([movies_metadata]) 
# ResultProxy = connection.execute(query)
# ResultSet = ResultProxy.fetchall()
# metadata_df = pd.DataFrame(ResultSet)
# metadata_df.columns = ResultSet[0].keys()
# metadata_df = metadata_df.loc[:, ['movie_id', 'release_date', 'runtime']]

# metadata_df.head()

In [8]:
cast = db.Table('movie_cast', metadata, autoload=True, autoload_with=engine)
query = db.select([cast])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
cast_df = pd.DataFrame(ResultSet)
cast_df.columns = ResultSet[0].keys()
cast_df = cast_df.loc[:, ['movie_id', 'cast_id']]

cast_df.head()

,movie_id,cast_id
0,1,31
1,2,2157
2,3,6837
3,4,8851
4,5,67773


In [9]:
# crew = db.Table('crew', metadata, autoload=True, autoload_with=engine)
# query = db.select([crew]) 
# ResultProxy = connection.execute(query)
# ResultSet = ResultProxy.fetchall()
# crew_df = pd.DataFrame(ResultSet)
# crew_df.columns = ResultSet[0].keys()
# crew_df = crew_df.loc[:, ['movie_id', 'name', 'job', 'gender']]

# crew_df.head()

In [10]:
connection.close()

In [11]:
# Set Index to Common Column of movie_id For Easier Join
movies_df = movies_df.set_index("movie_id")
ratings_df = ratings_df.set_index("movie_id")
mov_gen_df = mov_gen_df.set_index("movie_id")
# metadata_df = metadata.set_index("movie_id")
cast_df = cast_df.set_index("movie_id")
# crew_df = crew_df.set_index("movie_id")

In [14]:
data_df = ratings_df.join(movies_df, on="movie_id", how="left", sort=True)
data_df = data_df.join(mov_gen_df, on="movie_id", how="left", sort=True)
# data_df = data_df.join(metadata_df, on="movie_id", how="left", sort=True
data_df = data_df.join(cast_df, on="movie_id", how="left", sort=True)
# data_df = data_df.join(crew_gen_df, on="movie_id", how="left", sort=True
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id
movie_id,,,,,,,,
1,4,4.0,Toy Story,1995.0,81.0,0.0,1.0,31.0
1,4,4.0,Toy Story,1995.0,81.0,1.0,3.0,31.0
1,4,4.0,Toy Story,1995.0,81.0,2.0,4.0,31.0
1,4,4.0,Toy Story,1995.0,81.0,3.0,5.0,31.0
1,4,4.0,Toy Story,1995.0,81.0,4.0,9.0,31.0


In [15]:
data_df["movie_id"] = data_df.index
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id
movie_id,,,,,,,,,
1,4,4.0,Toy Story,1995.0,81.0,0.0,1.0,31.0,1
1,4,4.0,Toy Story,1995.0,81.0,1.0,3.0,31.0,1
1,4,4.0,Toy Story,1995.0,81.0,2.0,4.0,31.0,1
1,4,4.0,Toy Story,1995.0,81.0,3.0,5.0,31.0,1
1,4,4.0,Toy Story,1995.0,81.0,4.0,9.0,31.0,1


In [16]:
data_df = data_df.reset_index(drop=True)
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id
0,4,4.0,Toy Story,1995.0,81.0,0.0,1.0,31.0,1
1,4,4.0,Toy Story,1995.0,81.0,1.0,3.0,31.0,1
2,4,4.0,Toy Story,1995.0,81.0,2.0,4.0,31.0,1
3,4,4.0,Toy Story,1995.0,81.0,3.0,5.0,31.0,1
4,4,4.0,Toy Story,1995.0,81.0,4.0,9.0,31.0,1


In [17]:
data_df.count()

user_id     52632659
rating      52632659
name        52474568
year        52474568
runtime     52474568
mg_id       46951355
genre_id    46951355
cast_id     52104091
movie_id    52632659
dtype: int64

In [18]:
data_df = data_df.drop_duplicates(keep="first")
data_df = data_df.dropna()

In [19]:
data_df.count()

user_id     46631709
rating      46631709
name        46631709
year        46631709
runtime     46631709
mg_id       46631709
genre_id    46631709
cast_id     46631709
movie_id    46631709
dtype: int64

In [ ]:
data_df.to_csv("ML_Static_Dataframe_full.csv", index=False)